In [1]:
import tensorflow as tf
import tensorflow_decision_forests as tfdf
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt
import pandas as pd

import shap

In [2]:
wine_qt_path = '../data/WineQT.csv'
RANDOM_SEED = 492

In [3]:
wine_df = pd.read_csv(wine_qt_path)

In [5]:
print(wine_df.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0            7.4              0.70         0.00             1.9      0.076   
1            7.8              0.88         0.00             2.6      0.098   
2            7.8              0.76         0.04             2.3      0.092   
3           11.2              0.28         0.56             1.9      0.075   
4            7.4              0.70         0.00             1.9      0.076   

   free sulfur dioxide  total sulfur dioxide  density    pH  sulphates  \
0                 11.0                  34.0   0.9978  3.51       0.56   
1                 25.0                  67.0   0.9968  3.20       0.68   
2                 15.0                  54.0   0.9970  3.26       0.65   
3                 17.0                  60.0   0.9980  3.16       0.58   
4                 11.0                  34.0   0.9978  3.51       0.56   

   alcohol  quality  Id  
0      9.4        5   0  
1      9.8        5   1  
2      9

In [6]:
y_series = wine_df['Id']
y = pd.DataFrame(y_series, columns=['Id'])
features = [col for col in wine_df.columns if col != 'Id']
X = wine_df[features]

In [7]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_SEED)

In [8]:
X_train = X_train.reset_index(drop=True)
y_train = y_train.reset_index(drop=True)
X_test = X_test.reset_index(drop=True)
y_test = y_test.reset_index(drop=True)

In [9]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

X_train_scaled = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled = pd.DataFrame(X_test_scaled, columns=X_test.columns)

In [11]:
print(len(X_train_scaled))

914


In [10]:
subset_indices = np.random.choice(X_train.index, size=len(X_train), replace=False)

X_train_subset = X_train.loc[subset_indices]
y_train_subset = y_train.loc[subset_indices]

print(X_train_scaled.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0       0.105544         -1.379153     1.462888       -0.537610   0.087465   
1       0.621720         -1.046416     0.500012       -0.244623  -0.252294   
2       0.334956         -1.379153     0.905434       -0.610857  -0.412180   
3      -0.640043          0.229075    -1.121673       -0.171377  -0.132379   
4      -0.869454          0.173619    -1.223028       -0.610857  -0.072421   

   free sulfur dioxide  total sulfur dioxide   density        pH  sulphates  \
0             1.921184              1.727330  0.097116 -0.055335   0.537430   
1            -1.030261             -0.990201  0.667681 -0.564916  -0.225502   
2            -0.833498             -1.020395 -1.645700 -0.373823  -0.636312   
3             0.051936             -0.295721 -0.556440  0.454246  -0.342877   
4             0.248699             -0.325915  0.045247  0.836432  -0.166815   

    alcohol   quality  
0  0.042265  1.653380  
1 -1.146

In [17]:
subset_size = int(0.8 * len(X_train_scaled))
            
# Create a subset of the dataset using sampled indices
subset_indices = np.random.choice(X_train_scaled.index, size=subset_size, replace=False)
X_train_test4len = X_train_scaled.loc[subset_indices].reset_index(drop=True)
print(X_train_test4len.head())

   fixed acidity  volatile acidity  citric acid  residual sugar  chlorides  \
0       1.309954          1.670935     0.500012        0.048364  -0.032450   
1      -1.213571          0.672724    -1.070995       -0.684104  -0.751939   
2      -0.984160         -0.769136     1.108144       -0.610857  -0.352223   
3       1.080542          0.783637     0.601368       -0.464363  -0.671996   
4      -0.525337          0.506356    -1.121673       -0.464363  -0.392194   

   free sulfur dioxide  total sulfur dioxide   density        pH  sulphates  \
0             1.035750              0.730902  0.719551 -0.947101  -0.812374   
1            -0.931879             -0.839227 -1.676821  1.664501  -0.694999   
2             0.740606              3.116290 -2.340751 -1.201892  -0.929748   
3            -0.931879             -0.869421  0.460203 -0.883404  -1.105809   
4             0.838987             -0.114552 -0.317840  0.709037  -0.518938   

    alcohol   quality  
0 -0.506550 -0.850662  
1  1.368

In [12]:
X_train_tensor = tf.constant(X_train_subset.values, dtype=tf.float32)
y_train_tensor = tf.constant(y_train_subset.values, dtype=tf.float32)
print(y_train_tensor.shape)

(914, 1)


In [18]:
class GradientBoostedTreesEnsembleClassifier(tf.keras.Model):
    def __init__(self, n_trees=100, max_depth=3, n_estimators=100, subportion = 0.8, sample_method='RANDOM'): 
        super(GradientBoostedTreesEnsembleClassifier, self).__init__()
        self.n_trees = n_trees
        self.n_estimators = n_estimators
        self.max_depth = max_depth
        self.subportion = subportion
        self.sample_method = sample_method
        self.estimators = []

    def build_estimator(self):
        model = tfdf.keras.GradientBoostedTreesModel(
            task=tfdf.keras.Task.CLASSIFICATION,
            num_trees=np.random.randint(self.n_trees//5, self.n_trees+1), # self.n_trees, #
            max_depth=self.max_depth, #np.random.randint(self.max_depth/2, self.max_depth+1), # self.max_depth, # 
            validation_ratio=0.1,
            subsample=self.subportion,
            sampling_method=self.sample_method,
            random_seed=self.seed # try to set this first. Maybe this is enough
        )
        return model
    
    # if not, the number of trees/depth different for each estimator

    def fit(self, X, y):
        
        X_tensor = tf.constant(X.values, dtype=tf.float32)
        y_tensor = tf.constant(y.values, dtype=tf.float32)
        tot_dataset_size = len(X)
        
        @tf.function
        def train_single_estimator(seed):
            self.seed = int(seed)
            subset_size = int(self.subportion * tot_dataset_size)
            
            # Create a subset of the dataset using sampled indices
            subset_indices = np.random.choice(tot_dataset_size, size=subset_size, replace=False)
            
            X_subset = tf.gather(X_tensor, subset_indices)
            y_subset = tf.gather(y_tensor, subset_indices)
            
            # Building each estimator
            estimator = self.build_estimator()
            estimator.fit(X_subset, y_subset, verbose=0)
            self.estimators.append(estimator)
            
        for seed in np.random.randint(0, 1000000, size=self.n_estimators):
            train_single_estimator(seed)
    
    @tf.function
    def predict_mean(self, X): # use all estimators
        X_tensor = tf.constant(X.values, dtype=tf.float32)
        predictions = []
        
        for i, estimator in enumerate(self.estimators):
            estimator_predictions = estimator.predict(X_tensor)  # Get predictions from the current estimator
            print(f"Shape of prediction from estimator {i}: {estimator_predictions.shape}")  # Print the shape
            predictions.append(estimator_predictions)
        
        # predictions = [pred if len(pred.shape) == 2 else pred[:, np.newaxis] for pred in predictions]

        # Stack predictions for mean and standard deviation calculation
        # predictions_stacked = np.stack(predictions, axis=-1)
        
        
        # mean_prediction = np.mean(predictions_stacked, axis=-1)
        # sd_prediction = np.std(predictions_stacked, axis=-1)
        # return mean_prediction, sd_prediction
        return predictions


In [19]:
gbt_model = GradientBoostedTreesEnsembleClassifier(
    n_trees=50, 
    max_depth=5, 
    n_estimators=80, 
    subportion=0.8, 
    sample_method='RANDOM'
)

In [ ]:
gbt_model.fit(X_train_scaled, y_train)

Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmph4p2v238 as temporary training directory


[WARNING 24-06-05 18:23:33.3730 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:23:33.3737 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:23:33.3737 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 18:23:56.4142 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmph4p2v238/model/ with prefix d522cd1bf8fd4d02
[INFO 24-06-05 18:23:57.0110 CEST decision_forest.cc:734] Model loaded with 17578 root(s), 339034 node(s), and 12 input feature(s).
[INFO 24-06-05 18:23:57.0110 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 18:23:57.0111 CEST kernel.cc:1061] Use fast generic engine


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpa83fua_l as temporary training directory


[WARNING 24-06-05 18:23:59.3356 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:23:59.3357 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:23:59.3357 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 18:24:18.4261 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpa83fua_l/model/ with prefix 3dae0e753e6a4082


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmprf_21d9v as temporary training directory


...

[WARNING 24-06-05 18:33:52.0271 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:33:52.0272 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:33:52.0272 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 18:34:11.8590 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpe2ssuwqm/model/ with prefix 8656b48b6ec7483e


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpf9fhm_oj as temporary training directory


[INFO 24-06-05 18:34:12.4463 CEST decision_forest.cc:734] Model loaded with 17578 root(s), 333904 node(s), and 12 input feature(s).
[INFO 24-06-05 18:34:12.4464 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 18:34:12.4464 CEST kernel.cc:1061] Use fast generic engine
[WARNING 24-06-05 18:34:12.6319 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:34:12.6319 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:34:12.6319 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".
[INFO 24-06-05 18:34:32.8987 CEST kernel.cc:1233] Loading model from path /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpf9fhm_oj/model/ with prefix 13ee6bedc31249e6


Use /var/folders/3r/pzt2_p_x2xdfl2057nxkp3nm0000gn/T/tmpweam7b59 as temporary training directory


[INFO 24-06-05 18:34:33.4605 CEST decision_forest.cc:734] Model loaded with 17556 root(s), 313052 node(s), and 12 input feature(s).
[INFO 24-06-05 18:34:33.4605 CEST abstract_model.cc:1344] Engine "GradientBoostedTreesGeneric" built
[INFO 24-06-05 18:34:33.4605 CEST kernel.cc:1061] Use fast generic engine
[WARNING 24-06-05 18:34:33.6440 CEST gradient_boosted_trees.cc:1840] "goss_alpha" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:34:33.6440 CEST gradient_boosted_trees.cc:1851] "goss_beta" set but "sampling_method" not equal to "GOSS".
[WARNING 24-06-05 18:34:33.6440 CEST gradient_boosted_trees.cc:1865] "selective_gradient_boosting_ratio" set but "sampling_method" not equal to "SELGB".


In [ ]:
prediction = gbt_model.predict_mean(X_test_scaled)

In [34]:
mean_predict, sd_predict = gbt_model.predict_mean(X_test_scaled)

2024-06-05 16:00:20.332044: W tensorflow/core/framework/local_rendezvous.cc:404] Local rendezvous is aborting with status: INVALID_ARGUMENT: Shapes of all inputs must match: values[0].shape = [229,1598] != values[6].shape = [229,1596]


InvalidArgumentError: {{function_node __wrapped__Pack_N_80_device_/job:localhost/replica:0/task:0/device:CPU:0}} Shapes of all inputs must match: values[0].shape = [229,1598] != values[6].shape = [229,1596] [Op:Pack] name: stack

In [ ]:
y_test_eval = np.array(y_test).flatten()
y_pred_eval = mean_predict.numpy().flatten()

In [ ]:
PLOT = 30

In [ ]:
y_plot = y_test[:PLOT]
x_axis = np.arange(len(y_plot))
y_mean_predict = mean_predict[:PLOT].numpy()
y_var_predict = sd_predict[:PLOT].numpy()

In [ ]:
plt.figure(figsize=(8, 6))
plt.xlim(-1,PLOT)
plt.scatter(x_axis, y_plot, color='lightblue', label="Ground Truth")
plt.scatter(x_axis, y_mean_predict.flatten(), color='pink', alpha=0.8, label='Mean Prediction')
# plt.errorbar(x_axis, y_mean_predict.flatten(), yerr=root_mse, fmt='none', ecolor='pink', label="Root MSE")
plt.errorbar(x_axis, y_mean_predict.flatten(), yerr=y_var_predict.flatten(), fmt='none', color='darkred', alpha=0.8, label="Uncertainty")
plt.ylabel('prices')
plt.legend()
plt.grid(True)  # Enable grid for better visualization
plt.show()

In [ ]:
# predict gives the class number
# make a histogram of the class prediction
# entropy to measure the uncertainty